In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
files = ['lock_in.csv','tokyo.csv','vlr_2023.csv', 'madrid.csv', 'vlr_2024.csv', 'bangkok.csv','toronto.csv','vlr_2025.csv']
dfs = [pd.read_csv(f) for f in files]

df_all = pd.concat(dfs, ignore_index=True)

df_all["Player"] = df_all["Player"].str.strip()


In [3]:
# convert strings to nums
drop_cols = ['Agents']
int_cols = ['Rnd', 'K', 'D', 'A', 'FK', 'FD', 'KMax']
float_cols = ['R2.0', 'ACS', 'ADR', 'KPR', 'APR', 'K:D', 'FKPR', 'FDPR']
percent_cols = ['KAST', 'HS%', 'CL%', 'CL']

for col in int_cols:
    if col in df_all.columns:
        df_all[col] = pd.to_numeric(df_all[col], errors="coerce")

for col in float_cols:
    if col in df_all.columns:
        df_all[col] = pd.to_numeric(df_all[col], errors="coerce")

for col in percent_cols:
    if col in df_all.columns:
        df_all[col] = (
            df_all[col].str.replace("%", "", regex=False)
            .replace("", pd.NA)
        )
        df_all[col] = pd.to_numeric(df_all[col], errors="coerce") / 100.0

In [4]:
# Strip whitespace
df_all["Player"] = df_all["Player"].str.strip()

# Split into Player + Team using rpartition (safe & robust)
df_all["Player"], _, df_all["Team"] = zip(
    *df_all["Player"].apply(lambda x: x.rpartition(" "))
)

# Optional cleanup
df_all["Team"] = df_all["Team"].str.strip()
df_all["Player"] = df_all["Player"].str.strip()


In [11]:
# aggregate by all
agg_dict = {
    "Agents": "first",  # or you can build something fancier later
    "Team": "first",
    "Rnd": "sum",
    "R2.0": "mean",
    "ACS": "mean",
    "K:D": "mean",
    "KAST": "mean",
    "ADR": "mean",
    "KPR": "mean",
    "APR": "mean",
    "FKPR": "mean",
    "FDPR": "mean",
    "HS%": "mean",
    "CL%": "mean",
    "CL": "sum",
    "KMax": "max",  # or "max" if you prefer best single-series KMax
    "K": "sum",
    "D": "sum",
    "A": "sum",
    "FK": "sum",
    "FD": "sum",
}

player_agg = df_all.groupby("Player", as_index=False).agg(agg_dict)

# Weighted R2.0 by rounds played to downweight tiny events
weighted_r2 = (
    df_all.assign(weighted=df_all["R2.0"] * df_all["Rnd"])
    .groupby("Player", as_index=False)
    .agg(weighted_sum=("weighted", "sum"), rnd_sum=("Rnd", "sum"))
)
weighted_r2["R2.0_weighted"] = weighted_r2["weighted_sum"] / weighted_r2["rnd_sum"]
weighted_r2 = weighted_r2[["Player", "R2.0_weighted"]]

player_agg = player_agg.merge(weighted_r2, on="Player", how="left")
player_agg.insert(2, "R2.0_weighted", player_agg.pop("R2.0_weighted"))


In [12]:
# sort by R2.0_weighted
player_sorted = player_agg.sort_values(by="R2.0_weighted", ascending=False)

In [14]:


# Ensure ratings are numeric
player_agg["R2.0"] = pd.to_numeric(player_agg["R2.0"], errors="coerce")
player_agg["R2.0_weighted"] = pd.to_numeric(player_agg["R2.0_weighted"], errors="coerce")

# Sort & select top 20 (or 45 shown here)
top20 = (
    player_agg.sort_values(by="R2.0_weighted", ascending=False)
    .head(20)
    .reset_index(drop=True)
)

# Make a readable label like "Derke (FNATIC)"
top20["Label"] = top20["Player"] + " (" + top20["Team"] + ")"
# Text on bars: "1.23 (250 rnds)"
top20["TextLabel"] = top20.apply(
    lambda r: f"{r['R2.0_weighted']:.2f} ({int(r['Rnd'])} rnds)" if not np.isnan(r["R2.0_weighted"]) and not np.isnan(r["Rnd"]) else "",
    axis=1
)

fig = px.bar(
    top20,
    x="R2.0_weighted",
    y="Label",
    orientation="h",
    color="Team",
    text="TextLabel",  # show rating + rounds on the bar
    hover_data={
        "R2.0_weighted": ":.3f",
        "Rnd": True,
        "Team": True,
        "Label": False,  # hide from hover
    },
    title="Player R2.0 (weighted by rounds played)",
)

# Clean up visuals
fig.update_layout(
    xaxis_title="R2.0 (weighted by rounds)",
    yaxis_title="",
    showlegend=False,
    height=900,
)
fig.update_traces(textposition="outside", cliponaxis=False)

fig.show()


In [17]:
by_rating = player_agg.sort_values(by="R2.0_weighted", ascending=False)
pd.set_option('display.max_rows', None)
by_rating_filtered = by_rating[by_rating["Rnd"].ge(600)].reset_index(drop=True)
by_rating_filtered

,Player,Agents,R2.0_weighted,Team,Rnd,R2.0,ACS,K:D,KAST,ADR,...,FDPR,HS%,CL%,CL,KMax,K,D,A,FK,FD
0,Leo,None,1.278653,FNC,787,1.280000,210.666667,1.416667,0.810000,136.966667,...,0.020000,0.213333,0.206667,0.0,26,594,421,313,39,17
1,Demon1,(+1),1.170616,EG,958,1.123333,210.566667,1.200000,0.730000,142.200000,...,0.076667,0.386667,0.153333,0.0,28,803,632,165,129,84
2,Alfajer,(+1),1.146767,FNC,1890,1.160000,223.666667,1.291667,0.741667,149.016667,...,0.081667,0.303333,0.136667,0.0,29,1549,1230,336,231,159
3,aspas,(+1),1.146024,LOUD,1783,1.061667,220.600000,1.140000,0.715000,142.650000,...,0.133333,0.251667,0.154000,0.0,32,1493,1203,286,309,194
4,Less,(+1),1.141000,LOUD,1450,1.112000,212.980000,1.134000,0.722000,145.840000,...,0.088000,0.268000,0.148000,0.0,28,1143,970,360,133,114
5,RieNs,None,1.133729,TH,1019,1.085000,215.525000,1.072500,0.722500,144.050000,...,0.077500,0.340000,0.210000,0.0,28,776,688,307,87,79
6,Derke,None,1.132161,FNC,1305,1.130000,247.380000,1.220000,0.700000,161.220000,...,0.192000,0.234000,0.258000,0.0,32,1126,919,236,276,248
7,t3xture,None,1.108854,GE,794,1.085000,232.900000,1.175000,0.712500,148.375000,...,0.140000,0.252500,0.243333,0.0,27,677,566,132,144,112
8,iZu,(+1),1.107837,T1,601,1.085000,209.650000,1.165000,0.725000,127.700000,...,0.095000,0.280000,0.260000,0.0,22,454,380,118,69,55
9,kaajak,(+1),1.106755,FNC,872,1.105000,223.350000,1.165000,0.710000,144.850000,...,0.145000,0.335000,0.155000,0.0,32,699,601,156,174,124


In [ ]:
pd.reset_option('display.max_rows')